In [50]:
%matplotlib inline

import os
import re
import glob
import math
import pandas as pd
from natsort import natsorted
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

DEBUG = True

In [ ]:
print(os.getcwd())

In [52]:
def rel(path_str):
    return str((Path().resolve().parent / path_str).resolve())

PROFILES_FOLDER = rel('demo/2022-02/output/generator/profiles/cropped/')
RESULTS_FILE = rel('demo/2022-02/output/finder/finder.csv')

OUTPUT_FOLDER = rel('demo/2022-02/output/figures/figures')

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

In [53]:
def get_distance(ax, ay, bx, by):
    return ((ax - bx)**2 + (ay - by)**2) ** 0.5

def project_points(transect_csv, subpoints):
    subpoints_idx = []
    for subpoint in subpoints:

        dist1 = float('inf')
        dist2 = float('inf')
        for point in transect_csv.loc[:, ['x_geo', 'y_geo', 'no_point']].values:
            d1 = get_distance(point[0], point[1], subpoint[1], subpoint[2])
            d2 = get_distance(point[0], point[1], subpoint[3], subpoint[4])
            if d1 < dist1:
                dist1 = d1
                idx1 = point[2]
            if d2 < dist2:
                dist2 = d2
                idx2 = point[2]                
        
        subpoints_idx.append([min([idx1, idx2]), max([idx1, idx2])])

    return subpoints_idx

In [54]:
def draw_profile_features(title, axlines, features, points, transect, begin_no, end_no, zoom=False, save=True, show=True, link=False):
    
    cut = transect[transect['index_right'] >= 0]

    if len(cut) > 0:
        first_no = cut.iloc[0]['no_point']
        last_no = cut.iloc[-1]['no_point'] + 1
        first_no = first_no if first_no > begin_no else begin_no
        last_no = last_no if last_no < end_no else end_no
 
        x_min = begin_no if not zoom else first_no - 10
        x_max = end_no if not zoom else last_no + 10
            
        plt.xlim((x_min, x_max))
        max_y = max(transect.elevation) + 10  # Add a margin
        plt.ylim((min(transect.elevation) - 1, max(transect.elevation) + 10)) 
        #plt.gca().set_aspect('equal')
        plt.title(f'{title}  {"[zoomed]" if zoom else ""}')
        plt.xlabel('point no')
        plt.grid(linestyle=':')

        for line in axlines:
            plt.axhline(y = line, color = 'y', linestyle = '--')
            
        if link:
            point1 = [cut.iloc[0]['no_point'], transect.elevation[cut.iloc[0]['no_point']]]
            point2 = [cut.iloc[-1]['no_point'], transect.elevation[cut.iloc[-1]['no_point']]]
            x_values = [point1[0], point2[0]]
            y_values = [point1[1], point2[1]]
            plt.plot(x_values, y_values, 'r', linestyle="dotted")
        
        for f in features:
            y = transect[f][begin_no:end_no].to_numpy()
            plt.plot(np.arange(begin_no, end_no), y, '--')            
            
            y1 = transect[f][first_no:last_no].to_numpy()
            plt.plot(np.arange(first_no, last_no), y1)    
         
            
        def plot_point(name, point, base, color, mark):
            if math.isnan(point):
                return
    
            point = point.astype(int)
            if point < len(y) and point > 0:
                # point
                plt.plot(point, y[point], f'{color}{mark}', label=f'{name}: {point}')

                # line
                l = max(y) / 4
                yy = y[point] + (max((l, 4)) * base/abs(base)) # + label_y
                yy = min(yy, max_y - 1)
                plt.plot((point, point), (y[point], yy), f'{color}--')

                # label
                plt.annotate(point, (point, yy))

        if len(points) > 0:
            plot_point('zero', points['last_zero'].iloc[0], base=2, color='r', mark='o')
            plot_point('bottom', points['bottom'].iloc[0], base=2, color='b', mark='o')
            plot_point('top', points['top'].iloc[0], base=2, color='y', mark='o')

        plt.legend()
            
                

        fig = plt.gcf()

        if save:
            save_path = os.path.join(OUTPUT_FOLDER, f'{title}.cut{".zoom" if zoom else ""}.jpg')
            print(f"[SAVE] Saving to: {save_path}")
            fig.savefig(save_path, bbox_inches='tight', pad_inches=1, dpi=300)

        if show:
            print("[SHOW] Displaying figure...")
            display(fig)

        plt.close(fig)


In [55]:
csv_results = pd.read_csv(RESULTS_FILE, encoding='utf-8', sep=',', skipinitialspace=True)
id_list = list(map(int, csv_results.profile_id.dropna()))

In [ ]:
transect_files = list(
    filter(
        lambda p: int(os.path.basename(p).split('_')[0]) in id_list,
        natsorted(glob.glob(os.path.join(PROFILES_FOLDER, "*.csv")))
    )
)

transects = []
transect_names = []

for name in transect_files:
    transect_names.append(name)
    transects.append(pd.read_csv(name, encoding='utf-8', sep=',', skipinitialspace=True))

print(f"transect_files: {transect_files}")
print(f"transect_names: {transect_names}")
print(f"transects: {len(transects)}")
print(f"id_list: {list(id_list)}")

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
    
for idx in range(len(transects)):  ## for all profiles
#for idx in range(2, 7):   ## for selected profiles
    profile_id = int(os.path.basename(transect_names[idx]).split('_')[0])

    draw_profile_features(
        title=profile_id,
        axlines=[0, 20], # threshold
        features=['elevation'],
        points=csv_results.loc[csv_results['profile_id'] == profile_id],
        transect=transects[idx],
        begin_no=0,
        end_no=len(transects[idx]) - 1,        
        zoom=True,
        save=True,
        show=True,
        link=True
    )